In [25]:
import re
import os
import json
import time

import tweepy
import string
import datetime
import schedule
import urlextract
import pandas as pd

from html import unescape
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

In [26]:
def initiate_api():
    with open('./.conf/config.json', 'r') as f:
        config = json.load(f)
    auth = tweepy.OAuthHandler(config["CONSUMER_KEY"], config["CONSUMER_SECRET"])
    auth.set_access_token(config["ACCESS_KEY"], config["ACCESS_SECRET"])
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api

In [27]:
def collect_negative_tweets(api):
    
    query = '(🤬 OR 🤮 OR 😡 OR 😤 OR 🥺 OR 🤢 OR 😣 OR \
              😟 OR 😣 OR 🤔 OR 🤥 OR 😫 OR 🤮 OR 🥵 OR \
              😨 OR 😰 OR 😭 OR 😥 OR 🙁) \
        AND \
            -(😃 OR 😄 OR 😁 OR 🥰 OR 😊 OR ❤️ OR 💋 OR \
              😍 OR 😂 OR 😎 OR 🤣 OR 😘 OR 😇 OR 🙃 OR \
              😉 OR 😇 OR 🤩 OR 😃 OR 😄 OR 🙂 OR 😭)'
    
    tweets = []
    for status in tweepy.Cursor(api.search,
                           q=query,
                            include_entities=True,
                            monitor_rate_limit=True, 
                            wait_on_rate_limit=True,
                           lang="en").items(100):

        tweets.append([status.id_str, 
                       status.created_at, 
                       status.user.screen_name, 
                       status.text])
    return tweets

In [28]:
def save_raw_tweets(tweets):

    tweets_df = pd.DataFrame(tweets, 
                             columns=["ID", "Timestamp", "User", "Text"])
    
    now_prefix = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_")
    filepath = os.path.join("data","raw","tweets")
    
    if not os.path.exists(filepath):
        os.makedirs(filepath)
        
    filename = ''.join([now_prefix, "negative_tweets.csv"])
    
    tweets_df.to_csv(os.path.join(filepath, filename), 
                     index=False)

In [29]:
def negative_twitter_bot(api):
 
    try:
        tweets = collect_tweets(api)
    except:
        print("API limit exceeded. Waiting 1 hour.")
        time.sleep(3600)
        tweets = collect_tweets(api, query, date_since)
   
    save_raw_tweets(tweets)

In [32]:
def main():

    api = initiate_api()

    schedule.every(300).seconds.do(negative_twitter_bot, api)
    
    while True:
        schedule.run_pending()
        time.sleep(30)

In [ ]:
main()

API limit exceeded. Waiting 1 hour.
